In [ ]:
import re
import io
from IPython.display import display, Markdown
import pandas
from pandas import DataFrame
from numpy import nan, isnan
from ipywidgets import FileUpload, HBox, VBox, Output, Dropdown, Button, Layout
import plotly.express as px
import plotly.graph_objects as go
from typing import Union

pandas.set_option('mode.chained_assignment', 'raise')

# Constants
PREFLOP = 'preflop'
FLOP = 'flop'
RIVER = 'river'
TURN = 'turn'
SHOWDOWN = 'showdown'

SMALL_BLIND = 'small_blind'
BIG_BLIND = 'big_blind'
COLLECTED = 'collected'
JOINED = 'joined_with_stack'
QUIT = 'quit_with_stack'
BETS = 'bets'
RAISES = 'raises'
CALLS = 'calls'
CHECKS = 'checks'
FOLDS = 'folds'

SMALL_BLIND_MATCHER = re.compile('\"(.*)\" posts a small blind of (\d+)(?=\s|$)')
BIG_BLIND_MATCHER = re.compile('\"(.*)\" posts a big blind of (\d+)(?=\s|$)')

BETS_MATCHER = re.compile('\"(.*)\" bets (\d+)[\s\w]*')
RAISES_MATCHER = re.compile('\"(.*)\" raises to (\d+)[\s\w]*')
CALLS_MATCHER = re.compile('\"(.*)\" calls (\d+)[\s\w]*')
FOLDS_MATCHER = re.compile('\"(.*)\" folds$')
CHECKS_MATCHER = re.compile('\"(.*)\" checks$')
COLLECTED_MATCHER = re.compile('\"(.*)\" collected (\d+) from pot')

JOINED_MATCHER = re.compile('The player \"(.*?)\" joined the game with a stack of (\d+)\.')
QUIT_MATCHER = re.compile('The player \"(.*?)\" quits the game with a stack of (\d+)\.')


def prepare_poker_log(data: Union[bytes, str]) -> DataFrame:
    # Load CSV of Poker Now events
    poker_log: DataFrame = pandas.read_csv(data)
    poker_log = poker_log.drop(columns='order')
    # Reverse events so timestamps are in ascending order
    poker_log = poker_log[::-1].copy(deep=True)
    poker_log = poker_log.reset_index(drop=True)

    poker_log = poker_log.assign(hand_number=0, street='')

    starting_hand_rxp = re.compile("-- starting hand #(.*)  \(No Limit Texas Hold'em\) \((.*)\) --")

    ending_hand_rxp = re.compile("-- ending hand #(\d) --")

    # Hand Numbers
    curr_idx = 0
    while curr_idx < len(poker_log):
        sliced = poker_log.iloc[curr_idx:]

        # Find start index and end index of starting/ending hands
        # Since the PokerNow Event Log has showdowns between the end of one hand and the start of the next, we must track
        # both ending hand events and all events before the start of the next hand
        start_and_end = sliced.entry.str.startswith('-- starting hand').nlargest(2)
        # If there is no following

        # Starting Hand
        start_hand_idx = start_and_end.index.values[0]

        # Ending Hand Index
        end_hand_idx = sliced.entry.str.startswith('-- ending hand').idxmax()

        # Next Starting Hand
        if start_and_end.iloc[1]:
            next_start_idx = start_and_end.index.values[1]
        else:
            next_start_idx = len(poker_log)
        # Just before next starting hand
        end_idx = next_start_idx - 1

        street_slice = poker_log.iloc[start_hand_idx:end_idx+1]

        if street_slice.entry.str.startswith('Flop:').any():
            flop_idx = street_slice.entry.str.startswith('Flop:').idxmax()
            poker_log.loc[start_hand_idx:flop_idx, 'street'] = PREFLOP
            if street_slice.entry.str.startswith('Turn:').any():
                turn_idx = street_slice.entry.str.startswith('Turn:').idxmax()
                poker_log.loc[flop_idx:turn_idx, 'street'] = FLOP
                if street_slice.entry.str.startswith('River:').any():
                    river_idx = street_slice.entry.str.startswith('River:').idxmax()
                    poker_log.loc[turn_idx:river_idx, 'street'] = TURN
                    poker_log.loc[river_idx:end_hand_idx+1, 'street'] = RIVER
                    if end_idx != end_hand_idx:
                        poker_log.loc[end_hand_idx+1: end_idx+1, 'street'] = SHOWDOWN
                else:
                    poker_log.loc[turn_idx:end_idx+1, 'street'] = TURN
            else:
                poker_log.loc[flop_idx:end_idx+1, 'street'] = FLOP
        else:
            # No flop, skip others
            poker_log.loc[start_hand_idx:end_idx+1, 'street'] = PREFLOP

        # Parse hand number using regular expression
        start_match = re.match(starting_hand_rxp, sliced.entry[start_hand_idx])
        # Hand number
        hand_number = int(start_match.group(1))
        # Set hand numbers
        # If start of game or showdown occurred last hand, mark hand number appropriately
        # Else mark hand number as it was parsed from the regexp
        poker_log.loc[start_hand_idx:end_idx+1, 'hand_number'] = hand_number
        curr_idx = next_start_idx
        poker_log = poker_log.assign(player_name='', player_id='', action='', value='')

        poker_log.player_name = poker_log.entry.str.extract("\"(.*?) @ .*?\".*")
        poker_log.player_id = poker_log.entry.str.extract("\".*? @ (.*?)\".*")
        poker_log.player_name.fillna('')
        poker_log.player_id.fillna('')
        def parse_action_value(entry):
            if SMALL_BLIND_MATCHER.match(entry):
                return SMALL_BLIND, int(SMALL_BLIND_MATCHER.match(entry).group(2))
            elif BIG_BLIND_MATCHER.match(entry):
                return BIG_BLIND, int(BIG_BLIND_MATCHER.match(entry).group(2))
            elif BETS_MATCHER.match(entry):
                return BETS, int(BETS_MATCHER.match(entry).group(2))
            elif RAISES_MATCHER.match(entry):
                return RAISES, int(RAISES_MATCHER.match(entry).group(2))
            elif CALLS_MATCHER.match(entry):
                return CALLS, int(CALLS_MATCHER.match(entry).group(2))
            elif CHECKS_MATCHER.match(entry):
                return CHECKS, ''
            elif FOLDS_MATCHER.match(entry):
                return FOLDS, ''
            elif COLLECTED_MATCHER.match(entry):
                return COLLECTED, int(COLLECTED_MATCHER.match(entry).group(2))
            elif JOINED_MATCHER.match(entry):
                return JOINED, int(JOINED_MATCHER.match(entry).group(2))
            elif QUIT_MATCHER.match(entry):
                return QUIT, int(QUIT_MATCHER.match(entry).group(2))
            else:
                return '', ''
        poker_log['action'],  poker_log['value'] = zip(*poker_log['entry'].apply(parse_action_value))
        
    return poker_log
        

def calculate_cent_differential(poker_log: DataFrame):
    player_action_value_df = poker_log.copy(deep=True)
    player_action_value_df.replace('', nan, inplace=True)
    player_action_value_df.action.replace(JOINED, nan, inplace=True)
    player_action_value_df.dropna(subset=['action', 'value'], inplace=True)
    player_action_value_df.drop(columns='entry', inplace=True)

    """
    How total money won is calculated:

    Create 2d array to store result (row labels are winners, column labels are losers)

    First find the winner(s) of the hand (whoever collected is considered a winner)
    Then create a dict with the names of the participants per street -> last wager in a street
    Merge all street-specific dicts on name to get full contributions to pot per hand
    Remove winning name from dictionary.

    Iterate through dictionary and for each winner, add to 2d array how much each loser in that hand lost to the winner

    Subtract along the diagonal to get the differential for each player.
    """

    unique_players = poker_log.player_id.dropna().unique()
    unique_player_names = poker_log.player_name.dropna().unique()

    # One player can have multiple names but only one ID (many to one mapping). Use a dictionary to create a one-to-one mapping.
    name_id_dict = dict(zip(unique_players, unique_player_names))

    # Create 2d result array
    total_money_won = DataFrame(columns=unique_players, index=unique_players)
    total_money_won.fillna(0)
    hand_groups = player_action_value_df.groupby('hand_number', sort=False)
    for hand_number, hand_group in hand_groups:
        # Multiple winners if split pot
        winners = set(hand_group[hand_group['action'] == COLLECTED].player_id.values)
        loser_dict: dict = {}
        street_groups = hand_group.groupby('street')
        for street, street_group in street_groups:
            player_groups = street_group.groupby('player_id')
            for player, player_group in player_groups:
                if player not in winners:
                    last_play_value = player_group.tail(1).value.values[0]
                    if player in loser_dict:
                        loser_dict[player] += last_play_value
                    else:
                        loser_dict[player] = last_play_value
        for loser_name, value in loser_dict.items():
            # If split pot, divide by the number of winners
            split_value = value / len(winners)
            for winner_name in winners:
                if not isnan(total_money_won.loc[winner_name, loser_name]):
                    total_money_won.loc[winner_name, loser_name] = total_money_won.loc[winner_name, loser_name] + split_value
                else:
                    total_money_won.loc[winner_name, loser_name] = split_value
    # Ensure indexes and labels are in the same order
    total_money_won.sort_index(axis=0, inplace=True)
    total_money_won.sort_index(axis=1, inplace=True)
    total_money_won.fillna(0, inplace=True)

    total_money_won.rename(columns=name_id_dict, index=name_id_dict, inplace=True)


    total_money_difference = total_money_won.copy(deep=True)
    for i in range(0, len(total_money_won.columns)):
        for j in range(0, len(total_money_won)):
            total_money_difference.iloc[i, j] = total_money_won.iloc[i, j] - total_money_won.iloc[j, i]

    total_money_difference.rename(columns=name_id_dict, index=name_id_dict, inplace=True)
    total_money_difference.loc[:,'Total'] = total_money_difference.sum(axis=1)
    display(Markdown("## Total Cent Differential:"))
    display(total_money_difference)
    display(Markdown("## Total Cents Won: "))
    display(total_money_won)

def calculate_stacks_over_time(poker_log: DataFrame):
    # Get stacks over time (sot) by finding all 'Player stacks' events in event log
    sot = poker_log[poker_log.entry.str.startswith('Player stacks:')].copy(deep=True)
    sot.drop(columns=['street', 'action', 'value'], inplace=True)
    # Get all players starting and ending amounts using quit_with_stack and joined_with_stack events in event log
    joined_quit_log = poker_log[poker_log.action.eq(QUIT) | poker_log.action.eq(JOINED)].copy()
    joined_quit_log.drop(columns=['entry','street', 'action'], inplace=True)
    joined_quit_log.rename(columns={'value':'stack_size'},inplace=True)

    sot.entry = sot.entry.str.lstrip('Player stacks: ')
    sot = sot.assign(player_id='', stack_size=0)
    sot = sot.assign(split=sot.entry.str.split(' \| ')).explode('split')
    STACK_MATCHER = re.compile('#(\d+) \"(.*?) @ (.*?)\" \((\d+)\)$')

    def parse_stack_size(entry):
        match = STACK_MATCHER.match(entry)
        return match.group(3), int(match.group(4))

    # Create two columns for player id and stack size by parsing 'Player stacks' events
    sot['player_id'], sot['stack_size'] = zip(*sot['split'].apply(parse_stack_size))

    # Append starting and ending amounts to Player stacks events and sort by time stamp
    sot.drop(columns=['entry', 'split'], inplace=True)
    sot = sot.append(joined_quit_log)
    sot.sort_values(by='at', axis=0, inplace=True)
    sot = sot.reset_index(drop=True)

    unique_players = poker_log.player_id.dropna().unique()
    unique_player_names = poker_log.player_name.dropna().unique()

    # One player can have multiple names but only one ID (many to one mapping). Use a dictionary to create a one-to-one mapping.
    name_id_dict = dict(zip(unique_players, unique_player_names))

    sot.player_name = sot.player_id.map(name_id_dict)

    fig = px.line(sot, x="at", y="stack_size", color="player_name", title ='''Stack Size by Hand''',
            labels={'at':'Time','stack_size':'Chip Amount','player_name':'Player'}, hover_name='stack_size', hover_data=['stack_size', 'hand_number'], width=1000, height=600)
    fig.update_layout(
        title={
            'y':0.9,
            'x':0.5,
            'xanchor': 'center'})
    fig.update_traces(mode='markers+lines')
    fig.show()

ACTION_SORT_ORDER_KEY = {
    RAISES: 5,
    BETS: 4,
    CALLS: 3,
    CHECKS: 2,
    FOLDS: 1
}

def create_action_breakdown_charts(poker_log: DataFrame):

    actions = poker_log[poker_log.action.eq(BETS) | poker_log.action.eq(CALLS) | poker_log.action.eq(RAISES) | poker_log.action.eq(FOLDS) | poker_log.action.eq(CHECKS)].copy(deep=True)

    unique_players = poker_log.player_id.dropna().unique()
    unique_player_names = poker_log.player_name.dropna().unique()

    # One player can have multiple names but only one ID (many to one mapping). Use a dictionary to create a one-to-one mapping.
    name_id_dict = dict(zip(unique_players, unique_player_names))

    actions.player_name = actions.player_id.map(name_id_dict)
    actions.drop(columns=['entry'], inplace=True)

    grouped_actions = actions.groupby(['player_name', 'action']).size().unstack()
    grouped_actions = grouped_actions[sorted(grouped_actions.columns, key=lambda x: ACTION_SORT_ORDER_KEY[x])]

    action_breakdown = grouped_actions.apply(lambda x: x/sum(x), axis=1)
    action_breakdown.fillna(0, inplace=True)
    action_breakdown_bar_chart = px.bar(action_breakdown, title='Total Action Percentage Breakdown')
    action_breakdown_bar_chart.layout.legend.traceorder = 'reversed'
    action_breakdown_bar_chart.layout.yaxis.tickformat = ',.0%'
    action_breakdown_bar_chart.show()
    
    # Filter out anything other than types of bets
    bets = poker_log[poker_log.action.eq(BETS) | poker_log.action.eq(CALLS) | poker_log.action.eq(RAISES) | poker_log.action.eq(FOLDS) | poker_log.action.eq(CHECKS)].copy(deep=True)
    bets.player_name = bets.player_id.map(name_id_dict)

    fig = go.Figure()

    # Create a visibility mask for the plotly drop down menu
    # For each dropdown option, we must set a mask that selects only the graphs we want to show
    # Since certain streets may or may not have certain actions, we must create this in a slightly convoluted manner

    streets = ['all', PREFLOP, FLOP, TURN, RIVER]

    visibility_mask = [[], [], [], [], []]

    for i, street in enumerate(streets):
        if street == 'all':
            grouped = bets.groupby(['player_name', 'action']).size().unstack()
            
        else:                 
            grouped = bets[bets.street.eq(street)].groupby(['player_name', 'action']).size().unstack()

        grouped = grouped.fillna(0)
        grouped = grouped[sorted(grouped.columns, key=lambda x: ACTION_SORT_ORDER_KEY[x])]
        
        bar = px.bar(grouped)
        
        for j, street in enumerate(streets):
            # Create visibility mask
            if i == j:
                visibility_mask[j].extend(len(bar.data)*[True])
            else:
                visibility_mask[j].extend(len(bar.data)*[False])

        
        for trace in bar.data:
            # Add trace to final figure
            if street == 'all':
                fig.add_trace(go.Bar(trace, visible=True))
            else:
                fig.add_trace(go.Bar(trace, visible=False))

    fig.update_layout(
        updatemenus=[
            dict(
                active=0,
                buttons=list([
                    dict(label="Total",
                         method="update",
                         args=[{"visible": visibility_mask[0]},
                               {"title": "Total"}]),
                    dict(label="Preflop",
                         method="update",
                         args=[{"visible": visibility_mask[1]},
                               {"title": "Preflop"}]),
                    dict(label="Flop",
                         method="update",
                         args=[{"visible": visibility_mask[2]},
                               {"title": "Flop"}]),
                    dict(label="Turn",
                         method="update",
                         args=[{"visible": visibility_mask[3]},
                               {"title": "Turn"}]),
                    dict(label="River",
                         method="update",
                         args=[{"visible": visibility_mask[4]},
                               {"title": "River"}]),
                ]),
            )
        ])

    fig.update_layout(barmode='stack', legend={'traceorder':'reversed'}, title_text="Action By Street")

    fig.show()
    
    # Calculate how often a player raises versus any other action
    action_percent_no_fold = grouped_actions.drop(columns=[FOLDS]).apply(lambda x: x/sum(x), axis=1)
    action_percent_no_fold = action_percent_no_fold.assign(RaisesAndBets=action_percent_no_fold[RAISES]+action_percent_no_fold[BETS])
    raise_percent_bar_chart = px.bar(action_percent_no_fold.RaisesAndBets, title='# of Raises and Bets ÷ (# of Raises + # of Calls + # of Checks + # of Bets)')
    raise_percent_bar_chart.layout.yaxis.tickformat = ',.0%'
    raise_percent_bar_chart.show()

def calculate_hand_strength(poker_log: DataFrame):

    # Calculate percentile rank of all hands in a game
    player_hands = poker_log[poker_log.entry.str.startswith('Your hand is')].copy(deep=True)
    player_hands.drop(columns=['action','street','value','at', 'player_name', 'player_id'], inplace=True)
    player_hands.entry = player_hands.entry.str.lstrip('Your hand is ')
    player_hands = player_hands.assign(card=player_hands.entry.str.split(', '))

    CARD_RANKS = {
        'A':14,
        'K':13,
        'Q':12,
        'J':11,
        'T':10,
        '9':9,
        '8':8,
        '7':7,
        '6':6,
        '5':5,
        '4':4,
        '3':3,
        '2':2
    }


    def convert_hand_notation(hand):
        # Parse numbers and suits from cards in hand
        first_card = hand[0]
        first_number = str(first_card[:-1])
        first_suit = first_card[-1]
        second_card = hand[1]
        second_number = str(second_card[:-1])
        second_suit = second_card[-1]

        if first_number == '10':
            first_number = 'T'
        if second_number == '10':
            second_number = 'T'

        # Detect if hand is suited or off-suit
        if first_suit == second_suit:
            suited = 's'
        else:
            suited = 'o'

        # Sort cards in hand
        if CARD_RANKS[first_number] > CARD_RANKS[second_number]:
            return first_number + second_number + suited 
        else:
            return second_number + first_number + suited 


    player_hands = player_hands.assign(hand=player_hands.card.apply(convert_hand_notation))

    # Hand Rankings map of (Name -> Rank out of 169)
    hand_rankings_csv = pandas.read_csv('./data/preflop_hand_rankings.csv')

    hand_rankings_dict = dict(zip(hand_rankings_csv.Name, hand_rankings_csv.index))

    hand_ranks = (168 - player_hands.hand.map(hand_rankings_dict))/168
    hand_strength_percentile_rank = hand_ranks.mean()
    if isnan(hand_strength_percentile_rank):
        display('Please upload a log with hands.')
    else:
        display(Markdown('## Hand Strength Percentile Rank \n The following strength shows what percent of unique hands you beat on average. 50% means your hands were better than 50% of other hands on average.'))
        print('{:.1%}'.format(hand_strength_percentile_rank))

    hand_rankings_csv_freq = pandas.read_csv('./data/preflop_hand_rankings_frequency.csv')
    hand_rankings_dict_freq = dict(zip(hand_rankings_csv_freq.Name, hand_rankings_csv_freq.Rank))
    hand_ranks = (1326 - player_hands.hand.map(hand_rankings_dict_freq))/1326
    hand_luck_percentile_rank = hand_ranks.mean()
    if isnan(hand_luck_percentile_rank):
        display('Please upload a log with hands.')
    else:
        display(Markdown('## Hand Luck Percentile Rank: \n The following percent shows whether you were dealt better or worse than average hands in a given game.'))
        print('{:.1%}'.format(hand_luck_percentile_rank))
        
def submit(change):
    print('Loading. Please Wait.')
    if len(upload.data) > 0:
        data = io.BytesIO(upload.data[-1])
    else:
        data = dropdown.value
    poker_log = prepare_poker_log(data)  
    create_action_breakdown_charts(poker_log)
    calculate_cent_differential(poker_log)
    calculate_stacks_over_time(poker_log)
    calculate_hand_strength(poker_log)

upload = FileUpload(accept='.csv', multiple=False)

dropdown = Dropdown(
    options=[('Sample 1', './data/sample/poker_now_log_sample.csv'), ('Sample 2', './data/sample/poker_now_log_sample_2.csv')],
    value='./data/sample/poker_now_log_sample.csv',
    description='Or use: ',
)

go_button = Button(description='Go', layout=Layout(margin='auto', width='200px'))

go_button.on_click(submit)

vbox = VBox([HBox([upload, dropdown], layout=Layout(margin='20px auto')), go_button])
display(vbox)


